In [2]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.9.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


In [3]:
#Example 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai

n=2
m=2

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=x[2]#objective polynomial
g=[-x[1];x[1]-x[2]^2]#inequality constraints


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

2-element Vector{fmpq_mpoly}:
 -x_{1} - y_{1}^2
 x_{1} - x_{2}^2 - y_{2}^2

In [5]:
#compute the set h_FJ of polynomials from Fritz John conditions
h_FJ=Vector{fmpq_mpoly}(undef,n)
for i in 1:n
    h_FJ[i]=lamb[1]*derivative(f,i)
end
for j in 1:m
    for i in 1:n
        h_FJ[i]-=lamb[j+1]*derivative(g[j], i)
    end
end

for j in 1:m
    h_FJ=[h_FJ;lamb[j+1]*g[j]]
end

h_FJ=[h_FJ;1-sum(lamb.^2)]
h=[h;h_FJ]

h=[h;t[1]-f]#h=(gj(x)-yj^2,h_FJ,x_{n+1}-f)

13-element Vector{fmpq_mpoly}:
 -x_{1} - y_{1}^2
 x_{1} - x_{2}^2 - y_{2}^2
 lamb_{2} - lamb_{3}
 2*x_{2}*lamb_{3} + lamb_{1}
 -x_{1}*lamb_{2}
 x_{1}*lamb_{3} - x_{2}^2*lamb_{3}
 -lamb_{1}^2 - lamb_{2}^2 - lamb_{3}^2 + 1
 lamb_{2} - lamb_{3}
 2*x_{2}*lamb_{3} + lamb_{1}
 -x_{1}*lamb_{2}
 x_{1}*lamb_{3} - x_{2}^2*lamb_{3}
 -lamb_{1}^2 - lamb_{2}^2 - lamb_{3}^2 + 1
 -x_{2} + t_{1}

In [6]:
RI = radical(ideal(R, h))#compute radical of V(h)
#RI = ideal(R, h)
gens(RI)

8-element Vector{fmpq_mpoly}:
 t_{1}
 lamb_{2} - lamb_{3}
 lamb_{1}
 y_{2}
 y_{1}
 x_{2}
 x_{1}
 2*lamb_{3}^2 - 1

In [7]:
G=groebner_basis(RI,ordering=lex(gens(R)))

8-element Vector{fmpq_mpoly}:
 t_{1}
 2*lamb_{3}^2 - 1
 lamb_{2} - lamb_{3}
 lamb_{1}
 y_{2}
 y_{1}
 x_{2}
 x_{1}

In [9]:
check=1
alpha=zeros(Int64,1)
h_uni=Vector{fmpq_mpoly}([])
for j=1:length(G)
    check=1
    for i in 1:length(G[j])
        alpha=exponent_vector(G[j],i) 
        if sum(alpha[t] for t in setdiff(1:n+2*m+2,n+2*m+2))>0
            check=0
        end
    end
    if check==1
        h_uni=[h_uni;G[j]]
    end
end

h_uni# generators of the image f(S(g)\cap V(h_FJ))

1-element Vector{fmpq_mpoly}:
 t_{1}